In [1]:
import pandas as pd
import numpy as np

In [7]:
#!curl -L https://www.dropbox.com/s/4hct840aqr9drpl/mturk_surveys.csv?dl=1 -o mturk_surveys.csv


df = pd.read_csv("mturk_surveys.csv")
df["hit_answered_date"]=pd.to_datetime(df["hit_answered_date"])
df['date'] = pd.to_datetime(df['hit_answered_date'].apply(lambda x : x.date()))

# Create a list of all unique worker IDs 
workers = pd.DataFrame(list(set(df.worker_id)), columns=['worker_id']).reset_index().set_index('worker_id')
I = len(workers)

# Calculate the time period of each survey answer. 
# Below we take out time period to be 30-day periods. 
# We can change this to be weeks, months, or anything else
window_length = 30
minDate = min(df['date'])
df['period'] = (df['date'] - minDate).dt.days // window_length 

K = df['period'].max() + 1

data = df.pivot_table(
    index = 'worker_id',
    values = 'period',                    
    aggfunc=('count','min','max')
).join(workers)#.set_index('index').sort_index()

data.head(10)

,count,max,min,index
worker_id,,,,
00008f60155d83e5ac29d6942db234e7,4,31,23,27041
0000ceeb6c91d4a1a53fec0de0cc3460,1,1,1,3220
0001513be5c5e503366d994625ba169e,8,33,20,30621
0001760b118b1695f042928dc8d6eb5c,1,16,16,3616
000314db2adbfc1680937b7a9a7b0564,7,17,17,9475
00033655d33b221f842c11a00064cb05,1,11,11,28515
00038273373747793d09160ede6babdd,1,28,28,44703
000890517a0ee2c855083c5d2d41ad4f,1,0,0,41216
000922f9a9cf32952ea7b526fe655f07,3,19,14,33499


In [9]:
data[data['count'] > (data['max']-data['min'] + 1)].count()

count    609
max      609
min      609
index    609
dtype: int64

In [18]:
df.groupby(['worker_id', 'period']).size().sort_values(ascending=False).reset_index(name='count')

,worker_id,period,count
0,8e4ce5263b2b43ec70f532b6d58522f8,18,25
1,ad3089a4917de29f2ba115d2f31871cc,31,22
2,3569614c4966ce343374e6d38bc57ce4,32,19
3,7227d93f54edef858df51213e2da65f9,12,19
4,2c9f312b53b73d21c3e117b471ecad1d,32,16
5,8d581e715b2ca964b55dc6ec3a02f7bc,2,16
6,52629358bb4b600cd895af4db743e93a,9,15
7,d17c6ddf45e34036fbf534cb275cb734,15,14
8,b7613aa69fd695b7801ac1694d48af23,14,13
9,14da761c9066f3606778af1bce0176a2,7,13
